In [43]:
# Required packages
from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
%matplotlib inline

from PIL import Image
from IPython.display import display
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

print("Tensorflow version is", tf.__version__)

Tensorflow version is 1.8.0


In [44]:
def process_image(img:'np.array') -> 'np.array':
    """
        Input: Numpy Array
        Output: Numpy Array
        Takes an image as numpy array and returns processed image for prediction model.
    """
    try:
        exif = dict((ExifTags.TAGS[k], v) for k, v in img._getexif().items() if k in ExifTags.TAGS)
                    
        if exif['Orientation'] == 8:
            img = img.rotate(90, expand=True)
        elif exif['Orientation'] == 6:
            img = img.rotate(-90, expand=True)
        elif exif['Orientation'] == 3:
            img = img.rotate(180, expand=True)
    except:
        pass
    
    # Get the image dimensions
    w, h = img.size

    # Find the smaller dimension to crop the bigger dimension accordingly
    min_dim = min(w, h)

    # The number of pixels to be cropped would be half on both sides
    w_new = (w-min_dim) / 2
    h_new = (h-min_dim) / 2

    # Crop the image
    img = img.crop((w_new, h_new, w-w_new, h-h_new))

    # Resize the image
    size = (128, 128)
    img.thumbnail(size, Image.ANTIALIAS)
    
    # Convert to numpy array
    img_array = np.asarray(img, dtype='int32')
    
    # Normalize image
    img_array = img_array / np.max(img_array)
    
    # Reshape input image
    img_array = img_array.reshape((1, 128, 128, 3))
    
    return img_array

In [45]:
img_shape = (128, 128, 3)

base_model = tf.keras.applications.MobileNet(input_shape=img_shape, include_top=False, weights='imagenet')

In [46]:
model = tf.keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(2, activation='softmax')
])

In [47]:
model.load_weights('best_model.hdf5')

In [48]:
root = 'Sonali\'s phone'
list_of_images = os.listdir(root)

In [49]:
values = ['me', 'notme']

In [50]:
for image in list_of_images:
    try:
        img = Image.open(os.path.join(root, image))
    except:
        continue
    
    img.load()
    
    img2 = process_image(img)
    
    pred = model.predict(img2)
    
    pred = int(np.argmax(pred, axis=1))
    
    img.save(os.path.join('final', values[pred], image))